<a href="https://colab.research.google.com/github/aycaecemgul/InceptionV3_COVID-19/blob/master/Covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Jun  2 20:37:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [3]:
import os
import pickle
import random

import numpy as np
import tensorflow as tf
import tensorflow.keras
from PIL import Image
from matplotlib import pyplot as plt
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from numpy import asarray
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from numpy import asarray
from matplotlib import pyplot as plt

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!unrar x "/content/gdrive/My Drive/dataset.rar" "/content/gdrive/My Drive/DATASET/"

Görüntülenen çıkış son 5000 satıra kısaltıldı.
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_104.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_105.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_106.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_107.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_108.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_109.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_11.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_111.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_112.png      70%  OK 
Extracting  /content/gdrive/My Drive/DATASET/kaggle/1COVID/P029_113.png      70%  OK 
Extr

In [5]:
DIR = '/content/gdrive/MyDrive/DATASET/kaggle'
CATEGORIES = ["0NonCOVID","1COVID","2CAP"]
IMG_SIZE = 150
IMG_SHAPE=(150,150,3)

In [6]:
dataset_train=tf.keras.preprocessing.image_dataset_from_directory(
    DIR,
    labels="inferred",
    label_mode="int",
    class_names=CATEGORIES,
    color_mode="rgb",
    image_size=(IMG_SIZE,IMG_SIZE),
    shuffle=True,
    seed=123,
    validation_split=0.33,
    subset="training"
)

dataset_val=tf.keras.preprocessing.image_dataset_from_directory(
    DIR,
    labels="inferred",
    label_mode="int",
    class_names=CATEGORIES,
    color_mode="rgb",
    image_size=(IMG_SIZE,IMG_SIZE),
    shuffle=True,
    seed=123,
    validation_split=0.33,
    subset="validation"
)

Found 18215 files belonging to 3 classes.
Using 12205 files for training.
Found 18215 files belonging to 3 classes.
Using 6010 files for validation.


In [7]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [8]:
dataset_train = dataset_train.map(lambda x, y: (normalization_layer(x), y))
dataset_val = dataset_val.map(lambda x, y: (normalization_layer(x), y))

In [9]:
base_model = InceptionV3(
                          weights='imagenet',
                          include_top=False,
                          input_shape=IMG_SHAPE
                        )

base_model.trainable=False

model = tf.keras.Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()

87916544/87910968 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 3, 3, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 55299     
Total params: 21,858,083
Trainable params: 55,299
Non-trainable params: 21,802,784
_________________________________________________________________


In [10]:
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])


In [11]:
!pip install -q pyyaml h5py

In [13]:
# Checkpoint the weights when validation accuracy improves
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy
numpy.random.seed(123)

filepath="/content/gdrive/MyDrive/Training/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [14]:
model.fit(dataset_train,batch_size=32,validation_data = dataset_val,callbacks=callbacks_list,epochs = 15,verbose=2)

Epoch 1/15
382/382 - 1643s - loss: 1.4841 - accuracy: 0.7169 - val_loss: 1.1901 - val_accuracy: 0.7456

Epoch 00001: val_accuracy improved from -inf to 0.74559, saving model to /content/gdrive/MyDrive/Training/weights.best.hdf5
Epoch 2/15
382/382 - 47s - loss: 0.7623 - accuracy: 0.8228 - val_loss: 0.7746 - val_accuracy: 0.8101

Epoch 00002: val_accuracy improved from 0.74559 to 0.81015, saving model to /content/gdrive/MyDrive/Training/weights.best.hdf5
Epoch 3/15
382/382 - 40s - loss: 0.6002 - accuracy: 0.8510 - val_loss: 0.9084 - val_accuracy: 0.8403

Epoch 00003: val_accuracy improved from 0.81015 to 0.84027, saving model to /content/gdrive/MyDrive/Training/weights.best.hdf5
Epoch 4/15
382/382 - 39s - loss: 0.6588 - accuracy: 0.8537 - val_loss: 0.8467 - val_accuracy: 0.8309

Epoch 00004: val_accuracy did not improve from 0.84027
Epoch 5/15
382/382 - 47s - loss: 0.4591 - accuracy: 0.8889 - val_loss: 0.8411 - val_accuracy: 0.8656

Epoch 00005: val_accuracy improved from 0.84027 to 0.86

In [15]:
model.save("covid-19.model")

INFO:tensorflow:Assets written to: covid-19.model/assets


In [16]:
model.save("/content/gdrive/MyDrive/Training/covid-19.model")

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Training/covid-19.model/assets
